# Day 2

## Problem 1

Given a list of numbers, it passes if:
- either entirely decreasing or increasing
- each element differes by at most 3

In [1]:
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

with open('./data/day2_input.txt') as f:
    safe = 0

    for line in f:
        values  = list(map(int, line.split()))
        prev_dir = None
        is_safe = 1
        for vali, valj in zip(values, values[1:]):
            diff = vali - valj
            dist = abs(diff)
            dir = sign(diff)


            if dist > 3 or dist < 1 or (prev_dir is not None and dir != prev_dir):
                is_safe = 0
                break

            prev_dir = dir


        safe += is_safe
safe

660

## Problem 2

Now we have to account for removing a single level!

### First approach

This approach was wrong - full of (implicit) assumptions that were not true! Leaving here for reference.

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

def check_row_safety(vals,index_to_remove=None):
    values = vals


    if index_to_remove is not None:
        # Note here - we wouldnt have to start from the beginning of the list again if we just passed around the prev_dir!
        values = vals[:index_to_remove] + vals[index_to_remove+1:]
        level=2
        logger.debug(f"{' '*level}Now checking row safety for {values=} with {index_to_remove=} removed")
    else:
        logger.debug(f"START: checking row safety for {values=}")
        level=0
    
    prev_dir = None

    for idx, (vali, valj) in enumerate(zip(values, values[1:])):
            diff = vali - valj
            dist = abs(diff)
            dir = sign(diff)

            if dist > 3 or dist < 1 or (prev_dir is not None and dir != prev_dir):
                logger.debug(f"{' '*level}Row {values} is not safe! Returning index {idx+1}")
                return False, idx+1

            prev_dir = dir
            
    return True, None

unsafe_flipped = []
unsafe_flipped_last = []
unsafe = []
safes = []

with open('./data/day2_input.txt') as f:
    safe = 0

    for line in f:
        values  = list(map(int, line.split()))
        
        # check to see if the row is safe
        is_row_safe, failed_index = check_row_safety(values)

        # if not, try removing the failed index and checking again
        if not is_row_safe:
            if failed_index == len(values) - 1:
                logger.debug(f"  Last index is unsafe - counting this row as safe: {values}")
                is_row_safe = True
                unsafe_flipped_last.append(values)
            else:
                is_row_safe, _ = check_row_safety(values,failed_index)
                if is_row_safe:
                    logger.debug(f"  SAFE after removing index {failed_index}")
                    unsafe_flipped.append(values)
                else:
                    logger.debug(f"  STILL UNSAFE")
                    unsafe.append(values)
                
        else:
            # logger.debug(f"  Row is safe: {values}")
            safes.append(values)

        # collect safe rows
        safe += is_row_safe
    
safe

DEBUG:root:START: checking row safety for values=[27, 29, 30, 33, 34, 35, 37, 35]
DEBUG:root:Row [27, 29, 30, 33, 34, 35, 37, 35] is not safe! Returning index 7
DEBUG:root:  Last index is unsafe - counting this row as safe: [27, 29, 30, 33, 34, 35, 37, 35]
DEBUG:root:START: checking row safety for values=[51, 53, 54, 55, 57, 60, 63, 63]
DEBUG:root:Row [51, 53, 54, 55, 57, 60, 63, 63] is not safe! Returning index 7
DEBUG:root:  Last index is unsafe - counting this row as safe: [51, 53, 54, 55, 57, 60, 63, 63]
DEBUG:root:START: checking row safety for values=[87, 90, 93, 94, 98]
DEBUG:root:Row [87, 90, 93, 94, 98] is not safe! Returning index 4
DEBUG:root:  Last index is unsafe - counting this row as safe: [87, 90, 93, 94, 98]
DEBUG:root:START: checking row safety for values=[41, 42, 45, 47, 49, 51, 53, 58]
DEBUG:root:Row [41, 42, 45, 47, 49, 51, 53, 58] is not safe! Returning index 7
DEBUG:root:  Last index is unsafe - counting this row as safe: [41, 42, 45, 47, 49, 51, 53, 58]
DEBUG:ro

679

## Correct approach

After spending about an hour (off-and-on... dont judge) debugging and trying to figure out what was wrong... I realized that I was making a lot of assumptions that were not true trying to jump straight to the "optimized" solution. 

Lets take a particular example given to me in the input which is **safe** with the removal the first `8`:

```
[14, 11, 8, 9, 8, 6]
```

I assumed (implicitly!!!):
1. if a failure happens, we ONLY need to check the REMAINING elements and 1 before failure point - This was to prevent us having to recheck the entire list! I assumed it was safe until that point - so only removing the failure point and forward needed to be checked. In our example, it would mean we failed at 9, and then just needed to check `[8, 8, 6]` - which would fail, making this sequence unsafe. This is incorrect.
2. After figuring that out, I was SURE - if a failure happens, we only remove the failure point but check the entire row again. Still wrong.
3. This basically boils down to assuming that the initial failure point (right most) is always the reponsible one! This is incorrect! I should have considered the **pair** and checked if the removal of the left point and then the right point before passing or failing the sequence. This would have let me also catch the skipping the first element issue.
3. I assumed that if a failure happens at the last index, then we dont need to check it again - the sequence is safe. This is correct.
4. I assumed that since the first index could not be "responsible" for the failure, I never tried removing it - this goes with the assumption/realization mentioned above - should have thought about a failure pair.


### Code

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

def check_row_safety(vals,index_to_remove=None):
    values = vals

    if index_to_remove is not None:
        values = vals[:index_to_remove] + vals[index_to_remove+1:]
        level=2
        logger.debug(f"{' '*level}Now checking row safety for {values=} with {index_to_remove=} removed")
    else:
        logger.debug(f"START: checking row safety for {values=}")
        level=0
    
    prev_dir = None

    for idx, (vali, valj) in enumerate(zip(values, values[1:])):
            diff = vali - valj
            dist = abs(diff)
            dir = sign(diff)

            if dist > 3 or dist < 1 or (prev_dir is not None and dir != prev_dir):
                logger.debug(f"{' '*level}Row {values} is not safe! Returning index {idx+1}")
                return False, idx+1

            prev_dir = dir
            
    return True, None

unsafe_flipped2 = []
unsafe_flipped_last2 = []
unsafe2 = []
safes2 = []

with open('./data/day2_input.txt') as f:
    safe = 0

    for line in f:
        values  = list(map(int, line.split()))
        
        # check to see if the row is safe
        is_row_safe, failed_index = check_row_safety(values)

        # if not, try removing the failed index and checking again
        if not is_row_safe:
            for i in range(len(values)):
                is_row_safe, _ = check_row_safety(values[:i] + values[i+1:])
                if is_row_safe:
                    unsafe_flipped2.append(values)
                    break
            if not is_row_safe:
                unsafe2.append(values)
     
        else:
            # logger.debug(f"  Row is safe: {values}")
            safes2.append(values)

        # collect safe rows
        safe += is_row_safe
    
safe

DEBUG:root:START: checking row safety for values=[27, 29, 30, 33, 34, 35, 37, 35]
DEBUG:root:Row [27, 29, 30, 33, 34, 35, 37, 35] is not safe! Returning index 7
DEBUG:root:START: checking row safety for values=[29, 30, 33, 34, 35, 37, 35]
DEBUG:root:Row [29, 30, 33, 34, 35, 37, 35] is not safe! Returning index 6
DEBUG:root:START: checking row safety for values=[27, 30, 33, 34, 35, 37, 35]
DEBUG:root:Row [27, 30, 33, 34, 35, 37, 35] is not safe! Returning index 6
DEBUG:root:START: checking row safety for values=[27, 29, 33, 34, 35, 37, 35]
DEBUG:root:Row [27, 29, 33, 34, 35, 37, 35] is not safe! Returning index 2
DEBUG:root:START: checking row safety for values=[27, 29, 30, 34, 35, 37, 35]
DEBUG:root:Row [27, 29, 30, 34, 35, 37, 35] is not safe! Returning index 3
DEBUG:root:START: checking row safety for values=[27, 29, 30, 33, 35, 37, 35]
DEBUG:root:Row [27, 29, 30, 33, 35, 37, 35] is not safe! Returning index 6
DEBUG:root:START: checking row safety for values=[27, 29, 30, 33, 34, 37, 

689

### Final Clean Code

Cleaned up version of the above

In [7]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

def check_row_safety(vals,index_to_remove=None):
    values = vals
    if index_to_remove is not None:
        values = vals[:index_to_remove] + vals[index_to_remove+1:]
    
    prev_dir = None
    for idx, (vali, valj) in enumerate(zip(values, values[1:])):
            diff = vali - valj
            dist = abs(diff)
            dir = sign(diff)

            if dist > 3 or dist < 1 or (prev_dir is not None and dir != prev_dir):
                return False, idx+1

            prev_dir = dir
            
    return True, None

with open('./data/day2_input.txt') as f:
    safe = 0

    for line in f:
        values  = list(map(int, line.split()))
        
        # check to see if the row is safe
        is_row_safe, failed_index = check_row_safety(values)

        # if not, try removing the failed index and checking again
        if not is_row_safe:
            for i in range(len(values)):
                is_row_safe, _ = check_row_safety(values[:i] + values[i+1:])
                if is_row_safe:
                    break  
            
        # collect safe rows
        safe += is_row_safe
    
safe

689

# Notes

Of course we could just rewrite p1 and p2 using the same `check_row_safety` function, but thats less interesting than capturing the thought process and the mistakes made along the way.